# MonteCarlo

Importacion

In [ ]:
import random
import time
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import itertools
import pandas as pd

Aleatorios

In [ ]:
random.seed(int(time.time())) # semilla para replicar la aleatoriedad
random.random() # primer llamado a random

0.9973717654617937

Función para obtener el valor numérico de una carta

In [ ]:
def valor_carta(carta):
    if carta in ['J', 'Q', 'K']:
        return 10
    elif carta == 'A':
        return 11
    else:
        return int(carta)

Creación de las barajas

In [ ]:
def nueva_baraja(n_barajas):
    baraja_unica = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K'] * 4
    baraja_total = baraja_unica * n_barajas
    random.shuffle(baraja_total)
    return baraja_total

Se determina el valor de la mano dependiendo del valor preferible para las A´s

In [ ]:

def total(mano):
    total = sum(valor_carta(c) for c in mano)
    ases = mano.count('A')
    while total > 21 and ases:
        total -= 10
        ases -= 1
    return total


# **Simulación** de n juegos con las cartas dadas

In [ ]:


def simular_jugada(cartas_jugador, carta_crupier_visible, accion, barajas, n_sim=1000):
     ganancias = 0
     baraja = barajas.copy()
     for _ in range(n_sim):
        baraja = barajas.copy()
        random.shuffle(baraja)

        # Remover cartas ya en juego
        for c in cartas_jugador + [carta_crupier_visible]:
            if c in baraja:
                baraja.remove(c)

        jugador = cartas_jugador.copy()
        crupier = [carta_crupier_visible, baraja.pop()]

        # Acción del jugador
        if accion == "pedir":
            jugador.append(baraja.pop())
        elif accion == "doblar":
            jugador.append(baraja.pop())

        jugador_total = total(jugador)
        if jugador_total > 21:
            ganancias -= 2 if accion == "doblar" else 1
            continue

        # Juega el crupier
        while total(crupier) < 17:
            if len(baraja) < 1:
                continue  # No se puede pedir más
            crupier.append(baraja.pop())
        crupier_total = total(crupier)

        if jugador_total > crupier_total or crupier_total > 21:
            ganancias += 2 if accion == "doblar" else 1
           # ganancias += 1
        elif jugador_total == crupier_total:
            ganancias += 0
        else:
           ##ganancias -= 1
            ganancias -= 2 if accion == "doblar" else 1

     return ganancias / n_sim

In [ ]:
def obtener_mejor_opcion(cartas_jugador, carta_crupier, baraja):
    r_plantarse = simular_jugada(cartas_jugador, carta_crupier, "plantarse", baraja)
    r_pedir = simular_jugada(cartas_jugador, carta_crupier, "pedir",baraja)
    r_doblar = simular_jugada(cartas_jugador, carta_crupier, "doblar", baraja)

    mejor_accion = max(
        [("S", r_plantarse), ("H", r_pedir), ("D", r_doblar)],
        key=lambda x: x[1]
    )
    return mejor_accion

Ejecución de prueba

In [ ]:
r= simular_jugada([10,1], 1, "pedir", nueva_baraja(5))
print(r)

0.116


In [ ]:
def generar_tabla_decision_con_probabilidades(baraja):
    cartas_posibles = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
    combinaciones_jugador = list(itertools.combinations_with_replacement(cartas_posibles, 2))
    resultados = []

    for mano in combinaciones_jugador:
        fila = []
        for carta in cartas_posibles:
            r_plantarse = simular_jugada(list(mano), carta, "plantarse", baraja)
            r_pedir = simular_jugada(list(mano), carta, "pedir", baraja)
            r_doblar = simular_jugada(list(mano), carta, "doblar", baraja)

            mejor_accion = max([("S", r_plantarse), ("H", r_pedir), ("D", r_doblar)], key=lambda x: x[1])
            texto_celda = f"{mejor_accion[0]}:{mejor_accion[1]:.2f}"
            fila.append(texto_celda)
        resultados.append(fila)

    combinaciones_labels = [f"{a}-{b}" for a, b in combinaciones_jugador]
    graficar_heatmap_resultados(pd.DataFrame(resultados, index=combinaciones_labels, columns=cartas_posibles))
    return pd.DataFrame(resultados, index=combinaciones_labels, columns=cartas_posibles)

In [ ]:
df_resultados = generar_tabla_decision_con_probabilidades(n_barajas=6)

TypeError: generar_tabla_decision_con_probabilidades() got an unexpected keyword argument 'n_barajas'

In [ ]:
def graficar_heatmap_resultados(df_resultados):
    # Extraer valores numéricos para el heatmap
    mapa_valores = df_resultados.applymap(lambda x: float(x.split(":")[1]))

    sns.set_style("whitegrid")
    plt.figure(figsize=(18, 26))
    cmap = sns.color_palette("YlGnBu", as_cmap=True)

    ax = sns.heatmap(
        mapa_valores,
        annot=df_resultados,
        fmt='',
        cmap=cmap,
        linewidths=.5,
        linecolor='gray',
        vmin=0,
        vmax=1,
        cbar_kws={'label': 'Probabilidad de Ganar'}
    )

    ax.set_xlabel("Carta del Crupier")
    ax.set_ylabel("Cartas del Jugador")
    ax.set_title("Decisión óptima con probabilidad de ganancia\n(S=Plantarse, H=Pedir, D=Doblar)")
    plt.tight_layout()
    plt.show()


In [ ]:
df_resultados

NameError: name 'df_resultados' is not defined

In [ ]:
def capturar_y_jugar():
    import random

    print("=== Simulador de Partidas de Blackjack ===")

    while True:
        baraja = nueva_baraja(8)
        mano_jugador = []

        # Ingreso de cartas del jugador
        print("\nIngresa tus dos cartas iniciales:")
        for i in range(2):
            carta = input(f"  Carta {i+1}: ").strip().upper()
            mano_jugador.append(carta)

        # Carta del crupier
        carta_crupier = input("Ingresa la carta visible del crupier: ").strip().upper()

        while True:
            # Mostrar mano actual
            print(f"\nTu mano: {mano_jugador}")
            print(f"Carta del crupier: {carta_crupier}")

            # Evaluar la mejor opción actual
            try:
                opcion = obtener_mejor_opcion(mano_jugador, carta_crupier, baraja)
                print(f"Mejor opción según la estrategia: {opcion}")
            except Exception as e:
                print(f"Error al evaluar: {e}")
                break

            # Preguntar al jugador si desea otra carta
            decision = input("¿Deseas pedir otra carta? (S para sí, cualquier otra tecla para plantarte): ").strip().upper()
            if decision == "S":
                nueva_carta = input("Ingresa la nueva carta: ").strip().upper()
                mano_jugador.append(nueva_carta)
            else:
                break

        # Preguntar si quiere jugar otra partida
        otra = input("\n¿Quieres jugar otra partida? (S para sí, cualquier otra tecla para salir): ").strip().upper()
        if otra != "S":
            print("Gracias por jugar.")
            break


In [ ]:
# Ejecutar método
capturar_y_jugar()

=== Simulador de Partidas de Blackjack ===

Ingresa tus dos cartas iniciales:


KeyboardInterrupt: Interrupted by user

In [ ]:
def repartir_carta(baraja):
    return baraja.pop()

# Prueba de juego simulacion

In [ ]:
# ---------- Simulación de un juego ----------

def jugar_una_mano(apuesta, baraja):
    jugador = [repartir_carta(baraja), repartir_carta(baraja)]
    crupier = [repartir_carta(baraja), repartir_carta(baraja)]

    # Turno del jugador
    accion = obtener_mejor_opcion(jugador, crupier[0], baraja)

    while accion == "Hit":
        jugador.append(repartir_carta(baraja))
        if total(jugador) > 21:
            return -apuesta, accion  # Jugador pierde
        accion = obtener_mejor_opcion(jugador, crupier[0], baraja)

    # Turno del crupier
    while total(crupier) < 17:
        crupier.append(repartir_carta(baraja))

    valor_jugador = total(jugador)
    valor_crupier = total(crupier)

    if accion == "Double":
       apuesta *= 2

    if valor_crupier > 21 or valor_jugador > valor_crupier:
        return apuesta  # Jugador gana
    elif valor_jugador < valor_crupier:
        return -apuesta  # Jugador pierde
    else:
        return 0  # Empate


In [ ]:
# ---------- Simulación múltiple con gráfica ----------

def simular_juegos(n_juegos, apuesta=10, dinero_inicial=100, num=40):
    dinero = dinero_inicial
    historial_dinero = [dinero]
    baraja = nueva_baraja(8)
    fois=1

        # Contadores
    ganados = perdidos = empates = 0

    for i in range(n_juegos):

        if len(baraja) < num and fois == 1:  # por ejemplo, si quedan menos de 15 cartas
          fois=0;
          print("Cards left: ")
          print(len(baraja))
         # generar_tabla_decision_con_probabilidades(baraja)


        if len(baraja) < 15:  # por ejemplo, si quedan menos de 15 cartas
          baraja = nueva_baraja(8)

        resultado = jugar_una_mano(apuesta, baraja)
        dinero += resultado
        historial_dinero.append(dinero)

        # Clasificación de resultados
        if resultado > 0:
            ganados += 1
        elif resultado < 0:
            perdidos += 1
        else:
            empates += 1

 # Mostrar resultados
    print(f"\n--- Resultados después de {n_juegos} juegos ---")
    print(f"Ganados: {ganados} ({ganados / n_juegos * 100:.2f}%)")
    print(f"Perdidos: {perdidos} ({perdidos / n_juegos * 100:.2f}%)")
    print(f"Empates: {empates} ({empates / n_juegos * 100:.2f}%)")
    print(f"Dinero: {dinero}")


    # Gráfica
 #   plt.figure(figsize=(10,5))
  #  plt.plot(range(n_juegos + 1), historial_dinero, linestyle='-')
   # plt.title(f"Evolución del Dinero en {n_juegos} Juegos de Blackjack")
#    plt.xlabel("Número de Juegos")
 #   plt.ylabel("Dinero del Jugador")
  #  plt.grid(True)
   # plt.show()



In [ ]:
# ---------- Ejecutar Simulación ----------
simular_juegos(n_juegos=1000, apuesta=10, dinero_inicial=1000)
# ---------- Ejecutar Simulación ----------
simular_juegos(n_juegos=1000, apuesta=10, dinero_inicial=1000, num=30)
# ---------- Ejecutar Simulación ----------
simular_juegos(n_juegos=1000, apuesta=10, dinero_inicial=1000, num = 20)
# ---------- Ejecutar Simulación ----------
simular_juegos(n_juegos=1000, apuesta=10, dinero_inicial=1000, num = 52)


Cards left: 
39

--- Resultados después de 1000 juegos ---
Ganados: 382 (38.20%)
Perdidos: 566 (56.60%)
Empates: 52 (5.20%)
Dinero: -840
Cards left: 
27

--- Resultados después de 1000 juegos ---
Ganados: 388 (38.80%)
Perdidos: 564 (56.40%)
Empates: 48 (4.80%)
Dinero: -760
Cards left: 
17

--- Resultados después de 1000 juegos ---
Ganados: 373 (37.30%)
Perdidos: 586 (58.60%)
Empates: 41 (4.10%)
Dinero: -1130
Cards left: 
49

--- Resultados después de 1000 juegos ---
Ganados: 367 (36.70%)
Perdidos: 571 (57.10%)
Empates: 62 (6.20%)
Dinero: -1040
